# Baseline Modeling - Regression

---

* Goal: to develop baseline models prior to feature engineering to compare performance vs. post-engineered models.

My goal with this notebook is to develop a series of baseline models using minimal preprocessing. These models will establish a baseline performance for me to improve with additional feature engineering. Additionally, the most impactful features for each model can indicate if there are any features that are too strongly predictive.

---

In [ ]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import sweetviz as sv

In [ ]:
## SKLearn and Modeling Tools

from feature_engine.encoding import CountFrequencyEncoder, MeanEncoder
from feature_engine.outliers import Winsorizer
from feature_engine.pipeline import Pipeline as fePipeline

from sklearn import metrics
from sklearn import set_config
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, PowerTransformer, StandardScaler
from sklearn.tree import DecisionTreeRegressor

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

import tensorflow as tf

from xgboost import XGBRegressor, XGBRFRegressor

set_config(transform_output='pandas')

## Load Data

In [ ]:
df_data = pd.read_feather('../../data/source/full_data.feather')
# df_data = df_data.set_index('UUID')
df_data

## Set Target Feature

In [ ]:
target_feature = 'ADR'

## Quick Overview

In [ ]:
sv.analyze(df_data,pairwise_analysis = 'off').show_notebook()

---

**Quick Overview: Review**

Based on the quick EDA, I see there are both categorical features (several with high cardinality) and continuous (with right-tailed skews and some extreme outliers).

**Questionable Features**

First, I will drop the column `UUID` as it is a unique identifier and does not have any predictive value.

There are two features that I can identify from domain knowledge as being too strongly predictive of the ADR (`IsCanceled`, `ReservationStatus`). These features indicate whether or not a guest stayed (if they cancel or no-show, the revenue is zero).

Additionally, there are some temporal features that are either irrelevant to predictive modeling (`ArrivalDateYear`) or too closely related to the predictive features above (`ReservationStatusDate`).

I will drop these features to match real-world data more closely/realistically.

---

In [ ]:
df_data = df_data.drop(columns = ['UUID', 'IsCanceled',
                                  'ReservationStatus',
                                  'ReservationStatusDate',
                                  'ArrivalDateYear'])

In [ ]:
df_data.head().T

## Feature Selection

---

***We know too much!***

> The dataset contains several features which may not be knowable/available prior to the guest's stay.
> For the purposes of revenue management forecasting, I need either to drop the features prior to modeling or, as an "above and beyond" goal, create models to forecast each of the unknowable features using the pure baseline features.

**Features to Forget:**

> * Meal
> * IsRepeatedGuest
> * PreviousCancellations
> * PreviousBookingsNotCanceled
> * AssignedRoomType
> * BookingChanges
> * DaysInWaitingList
> * RequiredCarParkingSpaces
> * TotalOfSpecialRequests

---

In [ ]:
drop_feats = ['Meal','IsRepeatedGuest','PreviousCancellations',
              'PreviousBookingsNotCanceled','AssignedRoomType',
              'BookingChanges','DaysInWaitingList',
              'RequiredCarParkingSpaces','TotalOfSpecialRequests']

df_data = df_data.drop(columns = drop_feats)
df_data

# Outlier Inspection - Target

---

Based on my EDA, I noticed the target feature contains an extreme outlier of more than $5K. To improve my model performance, I will use the z-scores of each value to identify and remove outliers.

---

In [ ]:
df_data[target_feature].describe().round(2)

In [ ]:
df_data[target_feature].plot(kind = 'box')

In [ ]:
# Specify the column to filter
column_to_filter = 'ADR'  # Replace with the actual column name

# Calculate z-scores for the specified column
z_scores = np.abs(stats.zscore(df_data[column_to_filter]))

# Define a threshold for identifying outliers
threshold = 3

# Filter the data to remove outliers in the specified column
df_data = df_data[z_scores < threshold]
df_data.describe()

In [ ]:
df_data[target_feature].plot(kind = 'box')

# Train-Test Split

In [ ]:
df_data.head()

In [ ]:
X = df_data.drop(columns = target_feature)
y = df_data[target_feature]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 903)

# Create Modeling Pipelines

---

Prior to performing any modeling, I will need to perform some prep work - namely filling missing values; encoding categorical features; handling outliers in numeric features; and scaling features (mostly for linear regression models).

Before I fit my data using my preprocessing pipeline, I will transform my target feature using a Yeo-Johnson transformation. This will help reduce the impact of outliers in the target features without removing data entirely. Since scikit-learn's TransformedTargetRegressor class automatically inverts the transformation, the resulting predictions will be immediately interpretable.

Finally, I will evaluate my model's performance based on the R^2 score; mean absolute error ("MAE"); median absolute error ("MedAE"); and the root mean squared error ("RMSE"). These metrics will give me several perspectives on the model's performance.

---

# Random Forest Regressor Pipeline

In [ ]:
# Select categorical and numerical features
cat_feats = X.select_dtypes(include=['object']).columns
num_feats = X.select_dtypes(include=['number']).columns

## --- Create separate pipelines for categorical and numeric features --- ##

cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'most_frequent')),
                         ('encoder', CountFrequencyEncoder(unseen = 'encode',
                                                           encoding_method = 'frequency',
                                                           missing_values = 'ignore'))])

num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='mean')),
                         ('scaler', StandardScaler()),
                         ('powertransformer', PowerTransformer(method = 'yeo-johnson'))])

## --- Combine transformers into a single ColumnTransformer --- ##
preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, num_feats),
                                               ('cat', cat_pipeline, cat_feats)])

## ---  Create the TransformedTargetRegressor with Yeo-Johnson transformation --- ##
target_transformer = PowerTransformer(method='yeo-johnson')

base_regressor = RandomForestRegressor(n_jobs = -1)

regressor = TransformedTargetRegressor(regressor=base_regressor,
                                       transformer=target_transformer)

## --- Build the full pipeline --- ##
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('regressor', regressor)])

## --- Fit the model and generate predictions --- ##
model_pipeline.fit(X_train, y_train)

y_pred = model_pipeline.predict(X_test)

## --- Evaluate performance metrics --- ##
score_training = model_pipeline.score(X_train, y_train)
score_testing = model_pipeline.score(X_test, y_test)

print(f'\nTraining Score: {score_training:,.3f}\n'
      f'Testing Score: {score_testing:,.3f}\n'
      f'Difference: {score_training - score_testing:,.3f}\n')

mean_ae = metrics.mean_absolute_error(y_pred, y_test)
median_ae = metrics.median_absolute_error(y_pred, y_test)
mse = metrics.mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)


print(f'The MAE is ${mean_ae:,.2f}\n'
      f'The MedAE is ${median_ae:,.2f}\n'
      f'The RMSE is ${rmse:,.2f}\n')

## Model Tweaks

---



In [ ]:
depths = [tree.get_depth() for tree in model_pipeline[-1].regressor_.estimators_]

sns.histplot(depths);

In [ ]:
# Select categorical and numerical features
cat_feats = X_train.select_dtypes(include=['object']).columns
num_feats = X_train.select_dtypes(include=['number']).columns

# Create separate pipelines for categorical and numeric features
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', CountFrequencyEncoder(unseen='encode',
                                      encoding_method='frequency',
                                      missing_values='ignore'))
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('winsorizer', Winsorizer(capping_method='gaussian', tail='both', fold=3)),
    ('powertransformer', PowerTransformer(method='yeo-johnson'))
])

# Combine transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, num_feats),
    ('cat', cat_pipeline, cat_feats)
])

# Create the TransformedTargetRegressor with Yeo-Johnson transformation
target_transformer = PowerTransformer(method='yeo-johnson')

base_regressor = RandomForestRegressor(n_jobs=-1)

regressor = TransformedTargetRegressor(regressor=base_regressor,
                                       transformer=target_transformer)

# Build the full pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
])

# Set up hyperparameter tuning with HalvingGridSearchCV
param_grid = {
    'regressor__regressor__max_depth': [15,20],
    'regressor__regressor__min_samples_split': [2, 5, 10],
    'regressor__regressor__min_samples_leaf': [2, 4]
}

halving_grid_search = HalvingGridSearchCV(model_pipeline,
                                          param_grid,
                                        #   scoring='neg_median_absolute_error',
                                          cv=3,
                                          n_jobs=-1,
                                          factor=2,
                                          min_resources="exhaust")

# Fit the model and generate predictions
halving_grid_search.fit(X_train, y_train)

best_model = halving_grid_search.best_estimator_

y_pred = best_model.predict(X_test)

# Evaluate performance metrics
score_training = best_model.score(X_train, y_train)
score_testing = best_model.score(X_test, y_test)

print(f'\nTraining Score: {score_training:,.3f}\n'
      f'Testing Score: {score_testing:,.3f}\n'
      f'Difference: {score_training - score_testing:,.3f}\n')

mean_ae = metrics.mean_absolute_error(y_test, y_pred)
median_ae = metrics.median_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'The MAE is ${mean_ae:,.2f}\n'
      f'The MedAE is ${median_ae:,.2f}\n'
      f'The RMSE is ${rmse:,.2f}\n')

In [ ]:
# %%time

# # Select categorical and numerical features
# cat_feats = X.select_dtypes(include=['object']).columns
# num_feats = X.select_dtypes(include=['number']).columns

# ## --- Create separate pipelines for categorical and numeric features --- ##

# cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'most_frequent')),
#                          ('encoder', CountFrequencyEncoder(unseen = 'encode',
#                                                            encoding_method = 'frequency',
#                                                            missing_values = 'ignore'))])

# num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='mean')),
#                          ('powertransformer', PowerTransformer(method = 'yeo-johnson')),
#                          ('scaler', StandardScaler())])

# ## --- Combine transformers into a single ColumnTransformer --- ##
# preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, num_feats),
#                                                ('cat', cat_pipeline, cat_feats)])

# ## ---  Create the TransformedTargetRegressor with Yeo-Johnson transformation --- ##
# target_transformer = PowerTransformer(method='yeo-johnson')

# base_regressor = RandomForestRegressor()

# regressor = TransformedTargetRegressor(regressor=base_regressor,
#                                        transformer=target_transformer)

# ## --- Build the full pipeline --- ##
# model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                                  ('regressor', regressor)])

# ## --- Set up hyperparameter tuning with GridSearchCV --- ##
# param_grid = {
#       'regressor__regressor__max_depth': [25, 45],
#       'regressor__regressor__min_samples_split': [2, 3],
#       'regressor__regressor__min_samples_leaf': [2, 4]}

# grid_search = GridSearchCV(model_pipeline,
#                            param_grid,
#                            scoring='neg_median_absolute_error',
#                            cv=3,
#                            n_jobs=-1)

# ## --- Fit the model and generate predictions --- ##
# grid_search.fit(X_train, y_train)

# best_model = grid_search.best_estimator_

# y_pred = best_model.predict(X_test)

# ## --- Evaluate performance metrics --- ##
# score_training = best_model.score(X_train, y_train)
# score_testing = best_model.score(X_test, y_test)

# print(f'\nTraining Score: {score_training:,.3f}\n'
#       f'Testing Score: {score_testing:,.3f}\n'
#       f'Difference: {score_training - score_testing:,.3f}\n')

# mean_ae = metrics.mean_absolute_error(y_pred, y_test)
# median_ae = metrics.median_absolute_error(y_pred, y_test)
# mse = metrics.mean_squared_error(y_pred, y_test)
# rmse = np.sqrt(mse)


# print(f'The MAE is ${mean_ae:,.2f}\n'
#       f'The MedAE is ${median_ae:,.2f}\n'
#       f'The RMSE is ${rmse:,.2f}\n')

In [ ]:
print(f'The average actual ADR is ${y_test.mean().values[0].round(2)}.\n'
      f'''The model's predictions are off by about {round(median_ae/y_test.mean().values[0]*100,2)}%''')

## Model Inspection - Permutation Importances

---

In [ ]:
# Calculate permutation importances
result = permutation_importance(grid_search.best_estimator_,
                                X_test, y_test,
                                scoring = 'neg_median_absolute_error',
                                random_state=42,
                                n_jobs=-1)

# Extract importances and standard deviations
perm_importances = result.importances_mean
perm_importances_std = result.importances_std

# Create a DataFrame for easy plotting
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': perm_importances,
    'Importance_std': perm_importances_std
}).sort_values(by='Importance', ascending=False)

# Plot the feature importances
sns.barplot(x='Importance', y='Feature',data=importance_df);

## Save Model to Joblib

In [ ]:
joblib.dump(grid_search.best_estimator_,
            '../../data/baseline_model.joblib',
            compress = 9)

In [ ]:
saved_model = joblib.load('../../data/baseline_model.joblib')
saved_model

In [ ]:
saved_model.score(X_test, y_test)

# Decision Tree Regressor

In [ ]:
# Select categorical and numerical features
cat_feats = X_train.select_dtypes(include=['object']).columns
num_feats = X_train.select_dtypes(include=['number']).columns

# Create separate pipelines for categorical and numeric features
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', CountFrequencyEncoder(unseen='encode', encoding_method='frequency', missing_values='ignore'))
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('powertransformer', PowerTransformer(method='yeo-johnson')),
    ('scaler', StandardScaler())
])

# Combine transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, num_feats),
    ('cat', cat_pipeline, cat_feats)
])

# Create the TransformedTargetRegressor with Yeo-Johnson transformation
target_transformer = PowerTransformer(method='yeo-johnson')

base_regressor = DecisionTreeRegressor()

regressor = TransformedTargetRegressor(regressor=base_regressor, transformer=target_transformer)

# Build the full pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
])

# Set up hyperparameter tuning with HalvingGridSearchCV
param_grid = {
    'regressor__regressor__max_depth': [5, 10, 20, 30],
    'regressor__regressor__min_samples_split': [2, 5, 10],
    'regressor__regressor__min_samples_leaf': [2, 4, 8],
    'regressor__regressor__max_features': ['sqrt', 'log2', 0.5]
}

halving_grid_search = HalvingGridSearchCV(model_pipeline,
                                          param_grid,
                                          scoring='neg_median_absolute_error',
                                          cv=3,
                                          n_jobs=-1,
                                          factor=2,
                                          random_state=42)

# Fit the model and generate predictions
halving_grid_search.fit(X_train, y_train)

best_model = halving_grid_search.best_estimator_

y_pred = best_model.predict(X_test)

# Evaluate performance metrics
score_training = best_model.score(X_train, y_train)
score_testing = best_model.score(X_test, y_test)

print(f'\nTraining Score: {score_training:,.3f}\n'
      f'Testing Score: {score_testing:,.3f}\n'
      f'Difference: {score_training - score_testing:,.3f}\n')

mean_ae = metrics.mean_absolute_error(y_test, y_pred)
median_ae = metrics.median_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'The MAE is ${mean_ae:,.2f}\n'
      f'The MedAE is ${median_ae:,.2f}\n'
      f'The RMSE is ${rmse:,.2f}\n')

# DummyRegressor Pipeline

In [ ]:
# Define the target transformer
target_transformer = PowerTransformer(method='yeo-johnson')

# Instantiate the model
base_regressor = DummyRegressor()

# Create the TransformedTargetRegressor with Yeo-Johnson transformation
regressor = TransformedTargetRegressor(regressor=base_regressor,
                                       transformer=target_transformer)

# Build the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
])

# Fit the pipeline to the training data
model_pipeline.fit(X_train, y_train)

# Predict using the pipeline
y_pred = model_pipeline.predict(X_test)
score_training = model_pipeline.score(X_train, y_train)
score_testing = model_pipeline.score(X_test, y_test)

print(f'\nTraining Score: {score_training:,.3f}\n'
      f'Testing Score: {score_testing:,.3f}\n'
      f'Difference: {score_training - score_testing:,.3f}\n')

mean_ae = metrics.mean_absolute_error(y_pred, y_test)
median_ae = metrics.median_absolute_error(y_pred, y_test)
mse = metrics.mean_squared_error(y_pred, y_test)
rmse = np.log2(mse)


print(f'The MAE is ${mean_ae:,.2f}\n'
      f'The MedAE is ${median_ae:,.2f}\n'
      f'The RMSE is ${rmse:,.2f}\n')

# HistGradientBoostingRegressor

In [ ]:
## --- Create separate pipelines for categorical and numeric features --- ##

cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'most_frequent')),
                         ('encoder', CountFrequencyEncoder(unseen = 'encode',
                                                           encoding_method = 'frequency',
                                                           missing_values = 'ignore'))])

num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='mean')),
                         ('scaler', StandardScaler()),
                         ('powertransformer', PowerTransformer(method = 'yeo-johnson'))])

## --- Combine transformers into a single ColumnTransformer --- ##
preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, num_feats),
                                               ('cat', cat_pipeline, cat_feats)])

## ---  Create the TransformedTargetRegressor with Yeo-Johnson transformation --- ##
target_transformer = PowerTransformer(method='yeo-johnson')

base_regressor = HistGradientBoostingRegressor(loss = 'absolute_error')

regressor = TransformedTargetRegressor(regressor=base_regressor,
                                       transformer=target_transformer)

## --- Build the full pipeline --- ##
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('regressor', regressor)])

## --- Fit the model and generate predictions --- ##
model_pipeline.fit(X_train, y_train)

y_pred = model_pipeline.predict(X_test)

## --- Evaluate performance metrics --- ##
score_training = model_pipeline.score(X_train, y_train)
score_testing = model_pipeline.score(X_test, y_test)

print(f'\nTraining Score: {score_training:,.3f}\n'
      f'Testing Score: {score_testing:,.3f}\n'
      f'Difference: {score_training - score_testing:,.3f}\n')

mean_ae = metrics.mean_absolute_error(y_pred, y_test)
median_ae = metrics.median_absolute_error(y_pred, y_test)
mse = metrics.mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)


print(f'The MAE is ${mean_ae:,.2f}\n'
      f'The MedAE is ${median_ae:,.2f}\n'
      f'The RMSE is ${rmse:,.2f}\n')

In [ ]:
# Define the target transformer
target_transformer = PowerTransformer(method='yeo-johnson')

# Instantiate the model
base_regressor = HistGradientBoostingRegressor(random_state = 903)

# Create the TransformedTargetRegressor with Yeo-Johnson transformation
regressor = TransformedTargetRegressor(regressor=base_regressor,
                                       transformer=target_transformer)

# Build the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
])

# Fit the pipeline to the training data
model_pipeline.fit(X_train, y_train)

# Predict using the pipeline
y_pred = model_pipeline.predict(X_test)
score_training = model_pipeline.score(X_train, y_train)
score_testing = model_pipeline.score(X_test, y_test)

print(f'\nTraining Score: {score_training:,.3f}\n'
      f'Testing Score: {score_testing:,.3f}\n'
      f'Difference: {score_training - score_testing:,.3f}\n')

mean_ae = metrics.mean_absolute_error(y_pred, y_test)
median_ae = metrics.median_absolute_error(y_pred, y_test)
mse = metrics.mean_squared_error(y_pred, y_test)
rmse = np.sqrt(mse)


print(f'The MAE is ${mean_ae:,.2f}\n'
      f'The MedAE is ${median_ae:,.2f}\n'
      f'The RMSE is ${rmse:,.2f}\n')

# SGDRegressor

In [ ]:
# Define the target transformer
target_transformer = PowerTransformer(method='yeo-johnson')

# Instantiate the model
base_regressor = SGDRegressor(loss='huber',penalty='elasticnet')

# Create the TransformedTargetRegressor with Yeo-Johnson transformation
regressor = TransformedTargetRegressor(regressor=base_regressor,
                                       transformer=target_transformer)

# Build the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
])

# Fit the pipeline to the training data
model_pipeline.fit(X_train, y_train)

# Predict using the pipeline
y_pred = model_pipeline.predict(X_test)
score_training = model_pipeline.score(X_train, y_train)
score_testing = model_pipeline.score(X_test, y_test)

print(f'\nTraining Score: {score_training:,.3f}\n'
      f'Testing Score: {score_testing:,.3f}\n'
      f'Difference: {score_training - score_testing:,.3f}\n')

mean_ae = metrics.mean_absolute_error(y_pred, y_test)
median_ae = metrics.median_absolute_error(y_pred, y_test)
mse = metrics.mean_squared_error(y_pred, y_test)
rmse = np.log2(mse)


print(f'The MAE is ${mean_ae:,.2f}\n'
      f'The MedAE is ${median_ae:,.2f}\n'
      f'The RMSE is ${rmse:,.2f}\n')

# XGBRegressor

In [ ]:
# Define the target transformer
target_transformer = PowerTransformer(method='yeo-johnson')

# Instantiate the model
base_regressor = XGBRegressor(objective='reg:squarederror')

# Create the TransformedTargetRegressor with Yeo-Johnson transformation
regressor = TransformedTargetRegressor(regressor=base_regressor,
                                       transformer=target_transformer)

# Build the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
])

# Fit the pipeline to the training data
model_pipeline.fit(X_train, y_train)

# Predict using the pipeline
y_pred = model_pipeline.predict(X_test)
score_training = model_pipeline.score(X_train, y_train)
score_testing = model_pipeline.score(X_test, y_test)

print(f'\nTraining Score: {score_training:,.3f}\n'
      f'Testing Score: {score_testing:,.3f}\n'
      f'Difference: {score_training - score_testing:,.3f}\n')

mean_ae = metrics.mean_absolute_error(y_pred, y_test)
median_ae = metrics.median_absolute_error(y_pred, y_test)
mse = metrics.mean_squared_error(y_pred, y_test)
rmse = np.log2(mse)


print(f'The MAE is ${mean_ae:,.2f}\n'
      f'The MedAE is ${median_ae:,.2f}\n'
      f'The RMSE is ${rmse:,.2f}\n')

## XGBoost Random Forest Regressor

In [ ]:
# Define the target transformer
target_transformer = PowerTransformer(method='yeo-johnson')

# # Instantiate the model
# base_regressor = XGBRFRegressor(objective='reg:squarederror')

# Instantiate the model
regressor = XGBRFRegressor(objective='reg:squarederror')

# # Create the TransformedTargetRegressor with Yeo-Johnson transformation
# regressor = TransformedTargetRegressor(regressor=base_regressor,
#                                        transformer=target_transformer)

# Build the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
])

# Define parameter grid for GridSearchCV
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [3, 5, 7],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__subsample': [0.5, 0.7, 1.0],
    'regressor__colsample_bytree': [0.5, 0.7, 1.0]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(model_pipeline,
                           param_grid,
                           cv=5,
                           scoring='neg_median_absolute_error',
                           n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Predict using the pipeline
y_pred = grid_search.predict(X_test)
score_training = grid_search.score(X_train, y_train)
score_testing = grid_search.score(X_test, y_test)

print(f'\nTraining Score: {score_training:,.3f}\n'
      f'Testing Score: {score_testing:,.3f}\n'
      f'Difference: {score_training - score_testing:,.3f}\n')

mean_ae = metrics.mean_absolute_error(y_pred, y_test)
median_ae = metrics.median_absolute_error(y_pred, y_test)
mse = metrics.mean_squared_error(y_pred, y_test)
rmse = np.log2(mse)


print(f'The MAE is ${mean_ae:,.2f}\n'
      f'The MedAE is ${median_ae:,.2f}\n'
      f'The RMSE is ${rmse:,.2f}\n')

# Simple Neural Net

In [ ]:
X = df_data.drop(columns = target_feature)
y = df_data[target_feature]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 903)

In [ ]:
# Identify numerical and categorical features
num_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X_train.select_dtypes(include=['object']).columns.tolist()

cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'most_frequent')),
                         ('encoder', CountFrequencyEncoder(unseen = 'encode',
                                                           encoding_method = 'frequency',
                                                           missing_values = 'ignore'))])

num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='mean')),
                         ('powertransformer', PowerTransformer(method='yeo-johnson')),
                         ('scaler', MinMaxScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline, cat_features)
    ]
)

# Preprocess the features
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Perform Yeo-Johnson transformation on the target
target_transformer = PowerTransformer(method='yeo-johnson')
y_train_transformed = target_transformer.fit_transform(y_train.values.reshape(-1, 1))
y_test_transformed = target_transformer.transform(y_test.values.reshape(-1, 1))

# Convert data to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train_transformed = tf.convert_to_tensor(y_train_transformed, dtype=tf.float32)
y_test_transformed = tf.convert_to_tensor(y_test_transformed, dtype=tf.float32)

In [ ]:
# # Build the single-layer neural network model
# model = tf.keras.Sequential([
#     tf.keras.layers.InputLayer(shape=(X_train.shape[1],)),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(1)
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='mean_absolute_error')
# model.summary()

In [ ]:
def build_model():
        # Build the single-layer neural network model
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        # tf.keras.layers.Dense(16, activation='relu'),
        # tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    # Compile the model with the Adam optimizer and a learning rate scheduler
    initial_learning_rate = 0.001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=10000,
        decay_rate=0.9,
        staircase=True)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss='mean_absolute_error')
    model.summary()
    return model

In [ ]:
# # !pip install dojo-ds
# import dojo_ds as ds 
# help(ds.evaluate.plot_history)

In [ ]:
# Train the model
model = build_model()
history = model.fit(X_train, y_train, epochs=75, validation_split=0.2, verbose=1)

In [ ]:

# Evaluate the model
loss = model.evaluate(X_train, y_train, verbose=1)
print(f'Mean Absolute Error on Training Data: {loss:.2f}')

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=1)
print(f'Mean Absolute Error on Test Data: {loss:.2f}')

# Make predictions
y_pred = model.predict(X_test)

# Evaluate performance metrics
mean_ae = metrics.mean_absolute_error(y_test, y_pred)
median_ae = metrics.median_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'\nThe MAE is ${mean_ae:.2f}\n'
      f'The MedAE is ${median_ae:.2f}\n'
      f'The RMSE is ${rmse:.2f}\n')

# ds.evaluate.plot_history(history)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.plot(marker = 'o');

# Final Results

---

The best model was the Random Forest Regressor model. This model performed well with minor preprocessing, leading me to believe there may be features that are strongly predictive of the ADR. I will need to investigate further to confirm.

My next steps will be to perform additional EDA and feature engineering. During the EDA process, I will identify and remove any features that would not be known before the guest's stay. Then, I will engineer new features, including temporal features related to booking and stay dates, as well as hotel occupancy rates.

The end goal is to ensure my final dataset properly represents reservations prior to the guest stay and includes additional detail not present in the initial dataset.

---